# Chapter-12 : Networked programs

## Hypertext Transfer Protocol - HTTP

python integra un soporte integrado llamado **socket** para hacer conexiones de red y recuperar datos a travez de esos sockets mediante un programa en pyhton.

Un socket es muy parecido a un archivo, excepto que un solo socket proporciona una conexion bidireccional entre dos programas. puedes leer y escribir el mismo socket. Si escribes algo en un socket, se envia a la aplicación en el otro extremo del socket. y si leemos desde el socket, recibiras los datos que ha enviado la otra aplicación.

Sin intentamos leer datos de un socket  el otro extremo no ha enviado ningun dato, simplemente se sienta a esperar. Lo mas impotante de los programas qu se comunican por internet es tener algun protocolo.


## The world's simplest web browser

Quizas la forma mas simple de entender como funcionan los protocolos http, es esccribiendo un simple programa en python que se conecte a un servidor web y muestre lo que el servidor envia de vuelta.

In [1]:
import socket # importamos la libreria socket.

#
mySock = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 

# hacemos conexion por el puerto 80 en el servidor.
mySock.connect(('data.pr4e.org',80))

#               
cmd = 'GET http://data.pr4e.org/romeo.txt HTTP/1.0\r\n\r\n'.encode()

#               
mySock.send(cmd)
               
while True:
    data = mySock.recv(512)
    if len(data) < 1:
        break
    print(data.decode(), end = '')

mySock.close()    
                   
        

HTTP/1.1 200 OK
Date: Tue, 31 Mar 2020 23:34:26 GMT
Server: Apache/2.4.18 (Ubuntu)
Last-Modified: Sat, 13 May 2017 11:22:22 GMT
ETag: "a7-54f6609245537"
Accept-Ranges: bytes
Content-Length: 167
Cache-Control: max-age=0, no-cache, no-store, must-revalidate
Pragma: no-cache
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Connection: close
Content-Type: text/plain

But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief


## Retrieving an image over HTTP

In [2]:
import socket
import time

HOST = 'data.pr4e.org'
PORT = 80
mysock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
mysock.connect((HOST, PORT))
mysock.sendall(b'GET http://data.pr4e.org/cover3.jpg HTTP/1.0\r\n\r\n')
count = 0
picture = b""

while True:
    data = mysock.recv(5120)
    if len(data) < 1: break
    #time.sleep(0.25)
    count = count + len(data)
    print(len(data), count)
    picture = picture + data

mysock.close()

# Look for the end of the header (2 CRLF)
pos = picture.find(b"\r\n\r\n")
print('Header length', pos)
print(picture[:pos].decode())

# Skip past the header and save the picture data
picture = picture[pos+4:]
fhand = open("stuff.jpg", "wb")
fhand.write(picture)
fhand.close()


1448 1448
1448 2896
1448 4344
1448 5792
1448 7240
1448 8688
1448 10136
1448 11584
1448 13032
1448 14480
2896 17376
1448 18824
1448 20272
1448 21720
1448 23168
1448 24616
1448 26064
2896 28960
1448 30408
5120 35528
5016 40544
1448 41992
1448 43440
1448 44888
1448 46336
1448 47784
1448 49232
1448 50680
1448 52128
1448 53576
1448 55024
1448 56472
5120 61592
5120 66712
2792 69504
1448 70952
1448 72400
5120 77520
5120 82640
2792 85432
1448 86880
5120 92000
5120 97120
4240 101360
1448 102808
5120 107928
5016 112944
1448 114392
1448 115840
1448 117288
1448 118736
1448 120184
1448 121632
1448 123080
1448 124528
1448 125976
1448 127424
1448 128872
1448 130320
1448 131768
1448 133216
1448 134664
1448 136112
1448 137560
1448 139008
1448 140456
1448 141904
1448 143352
4344 147696
1448 149144
5120 154264
2120 156384
1448 157832
1448 159280
5120 164400
3568 167968
4344 172312
1448 173760
1448 175208
5120 180328
672 181000
1448 182448
1448 183896
1448 185344
1448 186792
1448 188240
1448 189688
1448 1

## Retrieving web pages with urllib

Usando la libreria **urllib** podemos tratar una pagina web como un archivo. simplemente  indicamos que pagina web desearia recuperar y **urllib** maneja todos los detalles del protocollo HTTP y de los encabezados.

El codigo equivalente para leer **romeo.txt** desde la web usando urllib es el siguiente: 

In [3]:
# importamos la libreria (el metodo .request)
import urllib.request

# indicamos la url de la web que deseamos recuperar.
fhand = urllib.request.urlopen('http://data.pr4e.org/romeo.txt')

for line in fhand:
    print(line.decode().strip())


But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief


Como ejemplo, podemos escribir un programa que recive la data de romeo.txt and cuente la frecuencia de cada palabra en el archivo.

In [4]:
# importamos los metodos de la libreria urllib
import urllib.request, urllib.parse, urllib.error

fhand = urllib.request.urlopen('http://data.pr4e.org/romeo.txt')

counts = dict()
for line in fhand:
    words = line.decode().split()
    for word in words:
        counts[word] = counts.get(word,0)+1

print(counts)        

{'But': 1, 'soft': 1, 'what': 1, 'light': 1, 'through': 1, 'yonder': 1, 'window': 1, 'breaks': 1, 'It': 1, 'is': 3, 'the': 3, 'east': 1, 'and': 3, 'Juliet': 1, 'sun': 2, 'Arise': 1, 'fair': 1, 'kill': 1, 'envious': 1, 'moon': 1, 'Who': 1, 'already': 1, 'sick': 1, 'pale': 1, 'with': 1, 'grief': 1}


## Reading binary files using urllib

Algunas veces queremos recivir un archivo non-text( o binario) como una imagen o video file. la data en estos casos no es util para imprimir, pero podemos facilmente hacer una copia de la url a un archivo local en nuestro disco duro con urllib.

El patron es abrir la url y usar leer para descargar todo el contenido del documento en una variable string (img) y luego escribir esa informacion en un archivo local de la siguiente manera:



In [5]:
import urllib.request, urllib.parse, urllib.error

img = urllib.request.urlopen('http://data.pr4e.org/cover3.jpg')
fhand = open('cover3.jpg', 'wb')
size = 0
while True:
    info = img.read(100000)
    if len(info) < 1: break
    size = size + len(info)
    fhand.write(info)


print(size, 'characters copied.')
fhand.close()

230210 characters copied.


## Parsing HTML and scraping the web

Una de los usos mas comunes de la libreria **urllib** es hacer scrape de la web. la web scraping es cuando escribimos un programa que pretende ser un web browser y recibir paginas, entonces examina los datos de estas paginas buscando modelos.



## Parsing HTML using regular expressions

Una de las formas para analizar HTML is usar expresiones regulares para buscar y extraer cadenas que coinciden con un patron en particular.

In [6]:
# Search for link values within URL input
import urllib.request, urllib.parse, urllib.error
import re
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Enter - ')
html = urllib.request.urlopen(url).read()
links = re.findall(b'href="(http[s]?://.*?)"', html)
for link in links:
    print(link.decode())


Enter - https://resultados.as.com/
https://resultados.as.com/resultados/
https://as.com
https://resultados.as.com/resultados/
https://as.com/futbol/
https://as.com/motor/
https://as.com/baloncesto/
https://as.com/tenis/
https://as.com/ciclismo/
https://as.com/opinion/
https://as.com/tag/mundial_2022/a/
https://esports.as.com/
https://as.com/ascolor/
https://as.com/videos/
https://as.com/tag/albumes/a/
https://as.com/diarioas/otras_webs_de_as.html
https://asfan.as.com/conectar?backURL=%2F%2Fresultados.as.com%2Fresultados%2F&prod=REG&o=CABAS
https://asfan.as.com/registro?backURL=%2F%2Fresultados.as.com%2Fresultados%2F&prod=REG&o=CABAS
http://arabia.as.com
https://asfan.as.com/conectar?backURL=%2F%2Fresultados.as.com%2Fresultados%2F&prod=REG&o=CABAS
https://asfan.as.com/registro?backURL=%2F%2Fresultados.as.com%2Fresultados%2F&prod=REG&o=CABAS
https://as.com/diarioas/2020/03/31/actualidad/1585628123_622265.html
https://as.com/diarioas/2020/03/31/actualidad/1585641693_897712.html
https://me

##  Parsing HTML using BeautifulSoup

In [7]:
#Importar las librerias.
import urllib.request, urllib.parse,urllib.error
from bs4 import BeautifulSoup
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Enter - ')
html = urllib.request.urlopen(url,context=ctx).read()
soup = BeautifulSoup(html,'lxml')

# Retrieve all of the anchor tags

tags = soup('a')
for tag in tags:
    print(tag.get('href',None))
    
    

Enter - https://resultados.as.com/
#menu
https://as.com
https://resultados.as.com/resultados/
https://as.com/futbol/
https://as.com/motor/
https://as.com/baloncesto/
https://as.com/tenis/
https://as.com/ciclismo/
//as.com/masdeporte/
https://as.com/opinion/
https://as.com/tag/mundial_2022/a/
https://esports.as.com/
https://as.com/ascolor/
https://as.com/videos/
https://as.com/tag/albumes/a/
https://as.com/diarioas/otras_webs_de_as.html
#resultados
//usuarios.as.com/newsletters/
#conecta
#buscador
#ediciones
https://asfan.as.com/conectar?backURL=%2F%2Fresultados.as.com%2Fresultados%2F&prod=REG&o=CABAS
https://asfan.as.com/registro?backURL=%2F%2Fresultados.as.com%2Fresultados%2F&prod=REG&o=CABAS
//as.com/?nrd=1
//chile.as.com/chile/?nrd=1
//colombia.as.com/colombia/?nrd=1
//mexico.as.com/mexico/?nrd=1
//us.as.com/us/?nrd=1
//argentina.as.com/argentina/?nrd=1
//peru.as.com/peru/?nrd=1
//as.com/diarioas/america.html?nrd=1
//en.as.com/en/?nrd=1
http://arabia.as.com
https://asfan.as.com/cone

In [35]:
# Web Scraping from Diario As.com
# Classification by team

from bs4 import BeautifulSoup
import requests 
import pandas as pd

# Cargar el html:

# URL donde se va hacer la operacion.
url = 'https://resultados.as.com/resultados/futbol/primera/clasificacion/'

# Descargar contenido de la pagina.
page = requests.get(url)

# Parse a forma to soup para poder identificar los
# diferentes formatos html
soup = BeautifulSoup(page.content,'html')

# Ahora identificamos los nombres de los equipos.
equipos = soup.find_all('span', class_ ='nombre-equipo')
# Ahora identificamos la puntuacion por equipos.
puntos = soup.find_all('td', class_ = 'destacado')

lista_equipos = list()
count = 0
for eq in equipos:
    if count < 20:
        lista_equipos.append(eq.text)
    else:
        break
    count +=1 
    
lista_puntos = list()
count = 0
for p in puntos:
    if count < 20:
        lista_puntos.append(p.text)
    else:
        break
    count +=1

# Creamos DataFrame Nombre, Puntos
df = pd.DataFrame({'Nombre':lista_equipos,'Putos':lista_puntos}, index=list(range(1,21)))
df

,Nombre,Putos
1,Barcelona,58
2,Real Madrid,56
3,Sevilla,47
4,R. Sociedad,46
5,Getafe,46
6,Atlético,45
7,Valencia,42
8,Villarreal,38
9,Granada,38
10,Athletic,37
